# 1. 출력 파서

   파서 | 출력 타입 | 특징 | 용도 |
:--- | :--- | :--- | :--- |
`StrOutputParser` | str | 가장 단순, AIMessage → 문자열 | 일반 텍스트 응답 |
`CommaSeparatedListOutputParser` | List[str] | 쉼표 구분 리스트 파싱 | 목록 생성 |
`JsonOutputParser` | dict | JSON 형식 파싱 | 구조화된 데이터 |
`PydanticOutputParser` | Pydantic 객체 | 타입 검증 포함 | 복잡한 스키마 |
`with_structured_output` | Pydantic/dict | 최신 권장 방식 | **구조화 출력** (권장) |

### StrOutputParser - 문자열 파싱

In [1]:
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# 프롬프트 정의

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 친절한 AI 어시스턴트입니다."),
    ("human", "{question}")
])

llm = init_chat_model("gpt-4o-mini")

chain = prompt | llm | StrOutputParser()

result = chain.invoke("파이썬의 장점 3가지를 알려주세요.")
print(result)
print(type(result))

파이썬의 장점은 여러 가지가 있지만, 특히 중요한 세 가지를 소개하겠습니다.

1. **읽기 쉬운 문법**: 파이썬은 간결하고 명확한 문법을 가지고 있어 코드를 쉽게 읽고 쓸 수 있습니다. 이는 초보자들이 배우기에 적합하며, 코드의 유지보수와 협업을 용이하게 합니다.

2. **광범위한 라이브러리와 프레임워크**: 파이썬은 다양한 분야에서 사용할 수 있는 풍부한 라이브러리와 프레임워크를 지원합니다. 예를 들어, 데이터 과학 분야에서는 NumPy, pandas, Matplotlib, TensorFlow와 같은 라이브러리를 사용할 수 있으며, 웹 개발 분야에서는 Django와 Flask 같은 프레임워크가 있습니다.

3. **다양한 커뮤니티와 지원**: 파이썬은 전 세계적으로 큰 커뮤니티를 가지고 있으며, 많은 자료와 튜토리얼, 포럼이 존재합니다. 이러한 커뮤니티의 지원은 문제 해결에 큰 도움이 되며, 다양한 리소스를 통해 학습할 수 있는 기회를 제공합니다.

이러한 장점들 덕분에 파이썬은 매우 인기 있는 프로그래밍 언어로 자리잡고 있습니다.
<class 'str'>


### CommaSeparatedListOutputParser - 리스트 파싱

In [3]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import init_chat_model

# 파서 생성
output_parser = CommaSeparatedListOutputParser()

# 포맷 지시사항 확인
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [4]:
# 프롬프트에 포맷 지시사항 포함
prompt = PromptTemplate(
    template="5가지 {subject}을(를) 나열하세요.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

# 체인 구성
llm = init_chat_model("gpt-4o-mini", temperature=0)
chain = prompt | llm | output_parser

# 실행
result = chain.invoke({"subject": "인기 있는 프로그래밍 언어"})
print(result)
print(type(result))

['Python', 'JavaScript', 'Java', 'C#', 'C++']
<class 'list'>


### JsonOutputParser - JSON 파싱
- Pydantic 모델을 사용하여 JSON 출력을 구조화합니다.

In [1]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import init_chat_model
from pydantic import BaseModel, Field

# Pydantic 모델 정의
class BookInfo(BaseModel):
    title: str = Field(description="책 제목")
    author: str = Field(description="저자 이름")
    year: int = Field(description="출판 연도")
    genre: str = Field(description="장르")

parser = JsonOutputParser(pydantic_object=BookInfo)

print(parser.get_format_instructions())

STRICT OUTPUT FORMAT:
- Return only the JSON value that conforms to the schema. Do not include any additional text, explanations, headings, or separators.
- Do not wrap the JSON in Markdown or code fences (no ``` or ```json).
- Do not prepend or append any text (e.g., do not write "Here is the JSON:").
- The response must be a single top-level JSON value exactly as required by the schema (object/array/etc.), with no trailing commas or comments.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]} the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema (shown in a code block for readability only — do not include any backticks or Markdown in your output):


In [3]:
# 프롬프트 구성
prompt = PromptTemplate(
    template="다음 책에 대한 정보를 제공해주세요: {book_name}\n{format_instructions}",
    input_variables=["book_name"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 체인 구성
llm = init_chat_model("gpt-4o-mini", temperature=0)
chain = prompt | llm | parser

# 실행
result = chain.invoke({"book_name": "해리포터와 마법사의 돌"})
print(result)

{'title': '해리포터와 마법사의 돌', 'author': 'J.K. 롤링', 'year': 1997, 'genre': '판타지'}


### with_structured_output (권장)
- LangChain 1.0에서 권장하는 구조화 출력 방식입니다. 모델의 네이티브 JSON 스키마 기능을 활용합니다.

**Pydantic 모델 사용(주로 사용됨)**

In [4]:
from langchain.chat_models import init_chat_model
from pydantic import BaseModel, Field

# 출력 스키마 정의
class MovieReview(BaseModel):
    """영화 리뷰 정보"""
    title: str = Field(description="영화 제목")
    rating: int = Field(description="1-10점 평점")
    pros: list[str] = Field(description="장점 목록")
    cons: list[str] = Field(description="단점 목록")
    summary: str = Field(description="한 줄 요약")

# 구조화 출력 설정
llm = init_chat_model("gpt-4o-mini")
structured_llm = llm.with_structured_output(MovieReview)

review = structured_llm.invoke("인셉션 영화를 리뷰해주세요.")

print(f"제목: {review.title}")
print(f"평점: {review.rating}/10")
print(f"장점: {review.pros}")
print(f"단점: {review.cons}")
print(f"요약: {review.summary}")

제목: 인셉션
평점: 9/10
장점: ['창의적인 스토리라인', '환상적인 비주얼 효과', '풍부한 캐릭터들', '강렬한 음악 사용', '지적인 작품으로 깊은 주제 다룸']
단점: ['복잡한 플롯으로 인한 혼란', '일부 관객에게는 이해하기 어려울 수 있음', '긴 러닝타임']
요약: 인셉션은 꿈 안의 꿈을 다루며 현실과 환상의 경계를 탐구하는 SF 액션 영화로, 크리스토퍼 놀란 감독의 정교한 연출과 뛰어난 비주얼이 매력적이다. 다층적인 스토리와 강렬한 캐릭터, 음악이 어우러져 관객에게 깊은 인상을 남긴다.


**TypedDict 사용**

In [3]:
from typing import TypedDict
from langchain.chat_models import init_chat_model

# TypedDict로 스키마 정의
class WeatherInfo(TypedDict):
    city: str
    temperatur: str
    condition: str
    humidity: str

# 구조화 출력 설정
llm = init_chat_model("gpt-4o-mini")
structured_llm = llm.with_structured_output(WeatherInfo)

# 실행
weather = structured_llm.invoke("서울의 현재 날씨를 알려주세요.")
print(weather)

{'city': '서울', 'temperatur': '15°C', 'condition': '맑음', 'humidity': '50%'}


**체인에서 파서 활용**

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field

# 1. 문자열 파서 체인
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{topic}에 대해 설명해주세요.")
    ]
)
llm = init_chat_model("gpt-4o-mini")

text_chain = prompt | llm | StrOutputParser()
result = text_chain.invoke({"topic": "양자 컴퓨팅"})

In [4]:
result

"양자 컴퓨팅(Quantum Computing)은 양자 역학의 원리를 바탕으로 정보를 처리하는 컴퓨터 시스템입니다. 전통적인 컴퓨터가 비트(bit)를 사용하여 정보를 0 또는 1의 두 가지 상태로 표현하는 반면, 양자 컴퓨터는 큐비트(qubit)를 사용합니다. 큐비트는 0과 1의 상태를 동시에 가질 수 있는 중첩(superposition) 상태를 가질 수 있습니다. 이로 인해 양자 컴퓨터는 특정 종류의 문제에 대해 훨씬 더 빠른 계산 능력을 제공합니다.\n\n양자 컴퓨터의 주요 개념은 다음과 같습니다:\n\n1. **큐비트(Qubit)**: 큐비트는 양자 컴퓨터의 기본 단위로, 두 가지 상태(0과 1)뿐만 아니라 이 두 상태의 중첩을 가질 수 있습니다. 큐비트의 수가 증가할수록 가능한 상태의 수가 기하급수적으로 늘어납니다.\n\n2. **중첩(Superposition)**: 큐비트는 0과 1의 상태를 동시에 가질 수 있습니다. 이로 인해 양자 컴퓨터는 여러 계산을 동시에 수행할 수 있는 잠재력을 가집니다.\n\n3. **얽힘(Entanglement)**: 얽힌 큐비트는 서로의 상태에 영향을 미치는 현상입니다. 한 큐비트의 상태가 변경되면, 다른 큐비트의 상태도 즉시 변경됩니다. 이를 통해 양자 컴퓨팅은 새로운 형태의 정보 처리 및 전송 방식이 가능합니다.\n\n4. **간섭(Interference)**: 양자 알고리즘은 중첩 상태의 간섭을 통해 유용한 결과를 도출하는 방식으로 디자인됩니다. 특정 경로의 확률을 높이거나 낮추는 방식으로 최적의 해답을 찾습니다.\n\n양자 컴퓨팅은 특정 문제에 대해 기존의 고전 컴퓨터보다 월등한 성능을 보일 수 있습니다. 대표적인 응용 분야로는:\n\n- **소인수 분해**: 쇼어 알고리즘(Shor's algorithm)을 통해 소인수 분해를 효율적으로 수행하여, 현재의 암호 체계에 대한 위협을 제기할 수 있습니다.\n- **최적화 문제**: 양자 컴퓨터는 복잡한 최적화 문제에서 더 빠른 해를 찾을 수 있는 잠재력을 가지고 있습니

In [ ]:
# 2. 구조화 출력 체인
class Explanation(BaseModel):
    concept: str = Field(description="개념 정의")
    examples: list[str] = Field(description="예시 목록")
    difficulty: str = Field(description="난이도: 초급/중급/고급")

structured_llm = llm.with_structured_output(Explanation)
structured_chain = prompt | structured_llm

result = structured_chain.invoke({"topic": "양자 컴퓨팅"})
print(f"개념: {result.concept}")
print(f"예시: {result.examples}")
print(f"난이도: {result.difficulty}")

개념: 양자 컴퓨팅은 양자역학의 원리를 기반으로 하는 컴퓨팅 기술로, 전통적인 컴퓨터가 사용하던 비트 대신 양자 비트(큐비트)를 사용하여 정보를 처리합니다. 큐비트는 0 또는 1의 상태뿐만 아니라 이 두 상태의 중첩(superposition) 상태를 가질 수 있어, 한 번에 여러 개의 계산을 동시에 수행할 수 있는 능력을 지니고 있습니다. 또한, 큐비트는 얽힘(entanglement)이라는 현상을 통해 서로 긴밀하게 연결될 수 있어, 복잡한 문제를 더 빠르게 해결할 수 있는 가능성을 제공합니다.
예시: ['양자 컴퓨터는 특정 수학 문제나 최적화 문제에서 전통 컴퓨터보다 훨씬 빠른 속도로 해결할 수 있습니다.', '양자 알고리즘 중 하나인 쇼어의 알고리즘은 정수를 소인수분해하는 데 전통적인 알고리즘보다 훨씬 빠른 해결책을 제공합니다.']
난이도: 중급
